In [1]:
import numpy as np

In [2]:
def read_data(file):
    with open(file, 'r') as fin:
        n = int(fin.readline().strip())
        data = []
        unique_labels = set()
        for i, line in enumerate(fin.readlines()):
            label, proba = map(float, line.strip().split())
            data.append((proba, label))
            unique_labels.add(label)
    probas, labels = zip(*sorted(data, reverse=True))
    unique_labels = sorted(list(unique_labels), reverse=True)
    return labels, probas, unique_labels

def calc_eq_sum(l):
    i = 0
    s = 0
    while i < len(l) - 1:
        s += l[i] * np.sum(l[i+1:])
        i += 1
    return s

def calc_delta_sum(lb_total, lb_equal, idx):
    return np.sum(lb_total[:idx] - lb_equal[:idx])

def multiclass_auc(labels, probas, unique_labels):
    
    labels_id = {}
    for i, l in zip(range(len(unique_labels)), unique_labels):
        labels_id[l] = i

    n_labels = len(unique_labels)
    lb_equal = np.zeros(n_labels)
    lb_total = np.zeros(n_labels)
    pr_prev = None
    a = 0

    for lb, pr in zip(labels, probas):
        if pr != pr_prev:
            a += calc_eq_sum(lb_equal) / 2
            lb_equal = np.zeros(n_labels)
            lb_equal[labels_id[lb]] = 1
            pr_prev = pr
        else:
            lb_equal[labels_id[lb]] += 1
        lb_total[labels_id[lb]] += 1
        a += calc_delta_sum(lb_total, lb_equal, labels_id[lb])
        #print(lb, pr, lb_total, lb_equal, a)

    a += calc_eq_sum(lb_equal) / 2

    b = 0
    for i in range(n_labels-1):
        cur = lb_total[i]
        b += cur * sum(lb_total[i+1:])

    return a / b

In [1]:
def main():
    labels, probas, unique_labels = read_data('random_test.txt')
    auc_score = multiclass_auc(labels, probas, unique_labels)
    fout = open("./output.txt", "w")
    fout.write("{:.6f}".format(auc_score))
    fout.close()
    print(auc_score)

In [4]:
main()